 # Sugar tutorial





The objective of Sugar library is to provide a set of handy tools to facilitate the setup of simulations of polyelectrolytes and peptides with the Molecular Dynamics software ESPResSo. In other words: adding sugar to your ESPResSo scripts makes your life sweater. In this tutorial, the different functionalities of sugar are explained with different practical examples.

## Introduction

Let us get started by importing sugar library and creating an instance of it. When sugar is inicialitated, the default system of reduced units is defined:

In [ ]:
import sugar
sg=sugar.sugar_library()

All input variables will be given to ESPResSo using these reduced units, since it is a convinient choice for the simulation setup. Internally, Sugar uses Pint library to deal with unit transformations, which in turn should be  used by the user to define its own variables

In [ ]:
Box_L = 15 * sg.units.nm
print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

However, the user is free to change the default equivalence to SI units to another of its own convienice, e.g.

In [ ]:
sg.set_reduced_units(unit_length=0.5*sg.units.nm,  unit_charge=2*sg.units.e, temperature=300 * sg.units.K)

The active set of reduced units can be consulted using

In [ ]:
sg.print_reduced_units()

## Use Sugar to create particles

The basic building block on Sugar library are particle objects, whose attributes define its chemical identity.

In [ ]:
cation = sg.particle()
cation.q = 1 # Charge of the particle
cation.type = 0 # Desired particle type to be loaded in espresso
cation.radius = 0.1 * sg.units.nm # Particle radius
cation.N = 5 # Desired number of particles
cation.name = 'Na' # label to the object

anion = sg.particle()
anion.q = - 1 
anion.type = 1 
anion.radius = 0.1 * sg.units.nm
anion.N = 5
anion.name = 'Cl'

This particle objects then can be used as input of several sugar functions. For instance, one can use sugar to create them into an espresso system

In [ ]:
import espressomd

system=espressomd.System(box_l=[Box_L.to('reduced_length').magnitude]*3)

sg.create_particle(system=system, particle=cation)
sg.create_particle(system=system, particle=anion)

for particle in system.part[:]:

    print('particle id', particle.id, 'particle type', particle.type,  'particle charge', particle.q)

All the information stored in the particle object (or any other sugar molecule object) can be easilly consulted 

In [ ]:
sg.write_parameters(mol=cation)

In the case of salt ions, these particle objects can be also used to create a given added salt concentration to your simulation. Please, note that in the current version of sugar only 1:1 salt can be created with these function.

In [ ]:
c_salt=1e-3*sg.units.mol/sg.units.L

c_salt_calculated=sg.create_added_salt(system=system, cation=cation, anion=anion, c_salt=c_salt)

## Use Sugar to create polyelectrolytes

Sugar uses particles as building blocks to create molecule objects with the following jerarchy. A molecule is composed of several residues, which in turn can have several particles. The structure of the molecule is given by the string variable `sequence`, where each letter encodes one residue name. For example, one can create a di-block strong polyampholyte as follows

In [ ]:
N_units=3

sequence='A'*N_units+'B'*N_units

# One needs to provide a dictionary with the desired atributes of each particle type in the molecule

particle_dict = {'A': {'type': 2,
                'q': -1,
                'radius': 0.2*sg.units.nm,
                            },
               'B': {'type': 3,
                'q': +1,
                'radius': 0.2*sg.units.nm,
                            }
              }

# One needs to provide the topology of the molecule in a dictionary. 
# The structure is principal_chain[residue_name] = particle_in_backbone_name
# If the keyword 'default' is residue_name, that particle is placed in the backbone unless otherwise specified
# If the keyword 'sequence' is particle_in_backbone_name, particles of the same type as defined in the sequence are used in the backbone

principal_chain={"default": "sequence"}

# With all this information you create your custom molecule model

custom_model=sg.create_custom_model(custom_particles=particle_dict, principal_chain=principal_chain)

# which you use to construct your molecule object

strong_polyampholyte=sg.molecule(sequence=sequence, param_custom=custom_model)

# that in turn you can use to create this molecules on your espresso system

sg.create_molecule(system=system, mol=strong_polyampholyte)

sg.write_parameters(strong_polyampholyte)


Similarly, one can create more complex molecules. Let us consider the case of an alternating weak polyampholyte (AB)_n, where we want to place the acidic (A) and basic groups (B) as side groups and use an inert group (C) as backbone 

In [ ]:
N_units=3

sequence='AB'*N_units

# For the acidic/basic particles, one needs to specify parameters 
# to define their protonated and unprotonated states

particle_dict = {'A':{'type':              
                           {'protonated':4,
                            'unprotonated':5},
                      'q': 
                           {'protonated':0,
                            'unprotonated':-1},
                    'radius': 0.2*sg.units.nm,
                    'acidity': 'acid',
                    'pKa': 4.25,
                    },
                'B':{ 'type': 
                           {'protonated':6,
                            'unprotonated':7},
                      'q': 
                           {'protonated':1,
                            'unprotonated':0},
                    'radius': 0.2*sg.units.nm,
                    'acidity': 'basic',
                    'pKa': 9,
                    },
               'C': {'type': 8,
                    'q': 0,
                    'radius': 0.2*sg.units.nm,
                    }
              }
            
# We specify that we want C particles as backbone

principal_chain={"default": "C"}

# We need to specify the topology of the side chains, the structure is
# side_chain={residue_name: [side_bead_name1, side_bead_name2, ]
# The keywords "default" and "sequence" can be also used with the same meaning as in principal_chain

side_chain={"default": ["sequence"]}

custom_model=sg.create_custom_model(custom_particles=particle_dict, 
                                    principal_chain=principal_chain,
                                    side_chain=side_chain
                                   )

weak_polyampholyte=sg.molecule(sequence=sequence, param_custom=custom_model)

weak_polyampholyte.N=2 # Desired number of polyampholytes in the system (by default 1)

sg.create_molecule(system=system, mol=weak_polyampholyte)

sg.write_parameters(weak_polyampholyte)

By default, harmonic bonds are used with an equilibrium bond length equal to the sum of the radii of the bonded particles and an harmonic constant of 300 `reduced_energy / reduced_length**2`. This values can be changed by the following commands

In [ ]:
sg.param.default.default_bond.bondl=sg.units.Quantity(1, 'reduced_length')
sg.param.default.default_bond.k=0.4 * sg.units.N / sg.units.m

Note that for these changes to be effective, one should change this values *before* calling `create_molecule()`. The parameters used to create each bond is stored in 'simulation_parameters.txt'

## Use Sugar to create peptides 

In contrast to polyelectrolytes, peptides have a highly heteregoneous structures usually composed of several different residues (aminoacids). If the framework presented in the previous section is used to construct a peptide, one would need to specify each aminoacid type in the peptide sequence. Instead, Sugar includes pre-built models that allow the user to quickly set-up peptides. For instance, one can construct a peptide using a one bead per aminoacid model

In [ ]:
sequence='nGGHIFGRTc' # Random example of a peptide sequence
model='1beadpeptide'

one_bead_peptide=sg.molecule(sequence=sequence, model=model)
sg.create_molecule(system=system, mol=one_bead_peptide)

sg.write_parameters(one_bead_peptide)

Note that in `sequence` we have used the one letter code for the aminoacids and we have denoted with 'n' and 'c' the amino and carboxilic ends, respectively. Alternatively, one can also construct a peptide using a model with 2 beads per aminoacids.

In [ ]:
# The same peptide as before, but now using the three letter code for aminoacids

sequence='NH2-GLY-GLY-HIS-ILE-PHE-GLY-ARG-THR-COOH'
model='2beadpeptide'
pKa_set='hass' 

two_bead_peptide=sg.molecule(sequence=sequence, model=model, pKa_set=pKa_set)

sg.create_molecule(system=system, mol=two_bead_peptide)
sg.write_parameters(two_bead_peptide)

By default, the pKa values of Ref. 1 (pKa_set='hass') are used for model = '1beadpeptide', '2beadpeptide'. However, Sugar includes other pre-built pKa sets, namely: "platzer" (Ref. 2), "crchandbook" (Ref. 3) or "nozaki" (Ref. 4). Values for the phosphorilated aminoacids J (pSER) U (pTHR) and Z (pTYR) are always taken from Ref. 5. Furthermore, the user can alter the pKa-value from one specific aminoacid from the set, or add a new one, by providing a custom set of pKa values

In [ ]:
pKa_dict={'n': 8.75, 'A': 4.25}

custom_two_bead_peptide=sg.molecule(sequence=sequence, model=model, pKa_set=pKa_set, pKa_custom=pKa_dict)

sg.create_molecule(system=system, mol=custom_two_bead_peptide)
sg.write_parameters(custom_two_bead_peptide)

By default, all particles are considered to have a radius equalt to 0.5 reduced_length, for both peptide models. Check `peptide_simulation_example.py` for an example script to simulate the titration of a peptide with Sugar and ESPResSo

## Use Sugar to setup simulations in ESPResSo

Sugar provides several functions to help the user to setup his simulation script in ESPResSo. For instance, the user can use his sugar molecule objects to create their counter-ions

In [ ]:
# Create counter-ions for the strong polyampholyte

positive_counterion, negative_counterion = sg.create_counterions(system=system, mol=strong_polyampholyte)

# Create counter-ions for the weak polyampholyte

positive_counterion, negative_counterion = sg.create_counterions(system=system, mol=weak_polyampholyte)

# Create counter-ions for the peptide using the one bead per aminoacid model

positive_counterion, negative_counterion = sg.create_counterions(system=system, mol=one_bead_peptide)

# Create counter-ions for the peptide using the two bead per aminoacid model

positive_counterion, negative_counterion = sg.create_counterions(system=system, mol=two_bead_peptide)

# Create counter-ions for the custom peptide peptide using the two bead per aminoacid model

positive_counterion, negative_counterion = sg.create_counterions(system=system, mol=custom_two_bead_peptide)

and to count the number of titrable groups in a molecule object

In [ ]:
N_titrable_polyampholyte=sg.count_titrable_groups(mol=weak_polyampholyte)

N_titrable_peptide=sg.count_titrable_groups(mol=one_bead_peptide)

N_peptide_chains=one_bead_peptide.N+two_bead_peptide.N+custom_two_bead_peptide.N
total_titrable_groups=N_titrable_polyampholyte*weak_polyampholyte.N+N_titrable_peptide*N_peptide_chains

Sugar can be also used to setup quickly all the acid-base reactions present in your molecule object to perform titration moves in the constant pH ensemble

In [ ]:
# import the reaction ensemble module from ESPResSo

import espressomd.reaction_ensemble

# Set-up of the acid/base reactions for the peptides (for simplicity, disregard the custom one)

RE_peptides=sg.setup_acidbase_reactions(mol=one_bead_peptide, counter_ion=cation)

# Set-up of the acid/base reactions for the weak polyampholyte

RE_polyampholyte=sg.setup_acidbase_reactions(mol=weak_polyampholyte, counter_ion=cation)

# Setup espresso to track the ionization of the acid/basic groups in the weak polyampholyte

sg.track_ionization(system=system, mol=weak_polyampholyte)

# Setup espresso to track the ionization of the acid/basic groups in the peptides

sg.track_ionization(system=system, mol=one_bead_peptide)

The sugar molecule/particle objects can also be used to quickly setup Lennard-Jones interactions between all the particles contained on them. By default, the Weeks-Chandler-Andersen (WCA) potential is used, with $\sigma = $ particle_1.radius + particle_2.radius, $\epsilon = 1$ reduced_energy and a cutoff radius of $2^{1/6} \sigma $. The specific interactions added for each particle type pair is stored in `simulation_parameters.txt`.

In [ ]:
molecule_list=[cation,
               anion, 
               positive_counterion, 
               negative_counterion,
               strong_polyampholyte,
               weak_polyampholyte,
               one_bead_peptide,
               two_bead_peptide,
               custom_two_bead_peptide]

sg.setup_lj_interactions(mol_list=molecule_list, system=system)

You can also setup electrostatic interactions in your system using Sugar. By default, p3m method is used but the Debye-Huckel potential is also implemented

In [ ]:
method='DH'

sg.setup_electrostatic_interactions(system=system, c_salt=c_salt, method=method)

Once all you have added all your interactions to your system, you can use sugar to minimize your system energy

In [ ]:
sg.minimize_system_energy(system=system)

and to setup the thermostat for Langevin Dynamics

In [ ]:
sg.setup_langevin_dynamics(system=system)

Note that in these processes many parameters are setup to default values that are reasonable choices in general but that should always be checked for your specific system. Please, refer to the documentation of each function for further details. Espresso visualizer can be used to check the molecules that we have created throught this tutorial

In [ ]:
from espressomd import visualization

visualizer = visualization.openGLLive(system, bond_type_radius=[0.3])

system.integrator.run(steps=1000) # to equilibrate the system

visualizer.run()

References

[1] Hass MA, Mulder FAA. Contemporary NMR Studies of Protein Electrostatics. Annu Rev Biophys. 2015;44:53-75.

[2] Platzer G, Okon M, McIntosh LP. 2014. pH-dependent random coil 1 H, 13 C, and 15 N chemical shifts of the ionizable amino acids: a guide for protein pK a measurements. J. Biomol. NMR 60:109–29

[3] Handbook of Chemistry and Physics, 72nd Edition, CRC Press, Boca Raton, FL, 1991.

[4] Y. Nozaki and C. Tanford, Methods Enzymol., 1967, 11, 715–734.

[5] Bienkiewicz & K.J. Lumb, J Biomol NMR 15: 203-206 (1999).